# **Analysing the factors of Laptop pricing**

## **Import Packages**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## **Load Data**

In [2]:
df = pd.read_csv("../data/laptop_price - dataset.csv")

## **Data Preparation**

### **Data Seperation As X and Y**

In [3]:
y = df["Price (Euro)"]
x = df.drop("Price (Euro)", axis=1)

### **Data spliting**

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## **Model Building**

### **Linear Regression**

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)